In [2]:
import pandas as pd
import numpy as np

In [3]:
county_fact = pd.read_csv(r'.\election\geo\county_facts.csv', index_col = 0)
#Minnesota no county info
# Alaska, District Of Columbia have duplicates, info not matched

county_fact.loc[county_fact['area_name'] == 'Alaska', 'state_abbreviation'] = 'AK'
county_fact.loc[county_fact['area_name'] == 'District Of Columbia', 'state_abbreviation'] = 'DC'
county_fact.loc[county_fact['area_name'] == 'District of Columbia', 'state_abbreviation'] = 'DC'
def capLetter(str1):
    tmp = str1.split(' ')
    for i in range(len(tmp)):
        x = tmp[i]
        if x =='of':
            pass
        else:
            tmp[i] = x[0].upper() + x[1:]
    res = tmp[0]
    if len(tmp)>1:
        for x in tmp[1:]:
            res = res + ' ' + x
    return res


county_fact['area_name'] = county_fact['area_name'].apply(capLetter)
county_fact.loc[county_fact['area_name'] == 'District of Columbia']

,area_name,state_abbreviation,PST045214,PST040210,PST120214,POP010210,AGE135214,AGE295214,AGE775214,SEX255214,...,SBO415207,SBO015207,MAN450207,WTN220207,RTN130207,RTN131207,AFN120207,BPS030214,LND110210,POP060210
fips,,,,,,,,,,,,,,,,,,,,,
11001,District of Columbia,DC,658893,601767,9.5,601723,6.5,17.5,11.3,52.6,...,6.1,34.5,332844,2117990,3843716,6555,4278171,4189,61.05,9856.6


In [4]:
def clean_tail(str1):
    
    try:
        tmp = str(str1).split('[')[0]
    except:
        tmp = str1
    if tmp == 'nan':
        tmp = ''
    return tmp

In [5]:
cleaned_dems = pd.read_csv(r'.\election\primary_data\cleaned_dems.csv', parse_dates = ['Date'])
cleaned_dems['State'] = cleaned_dems['State/territory']
del cleaned_dems['State/territory']
del cleaned_dems['wrongDate']
for col in cleaned_dems.columns:
    cleaned_dems[col] = cleaned_dems[col].apply(clean_tail)
cleaned_dems_2 = cleaned_dems.merge(county_fact, how = 'inner', left_on = 'State', right_on = 'area_name')
cleaned_dems_2.to_csv(r'.\stitched_data\states_dems.csv', index = False)

In [6]:
cleaned_reps = pd.read_csv(r'.\election\primary_data\cleaned_reps.csv', parse_dates = ['Date'])
for col in cleaned_reps.columns:
    cleaned_reps[col] = cleaned_reps[col].apply(clean_tail)
cleaned_reps_2 = cleaned_reps.merge(county_fact, how = 'inner', left_on = 'State', right_on = 'area_name')
cleaned_reps_2.to_csv(r'.\stitched_data\states_reps.csv', index = False)

In [7]:
state_abbr = pd.read_csv(r'.\election\geo\state_abbr_yuan.csv')
#state_abbr['State'] = state_abbr['State'].apply(lambda x: x[0]+x[1:].lower())

In [11]:
primary = pd.read_csv(r'.\election\primary_data\primary_processed.csv', index_col = 0)
del primary['Unnamed: 0.1']
primary['edge1'] = np.nan
primary['edge2'] = np.nan
primary['edge3'] = np.nan
for idx,row in primary.iterrows():
    if row['3rd'] in ['UNCOMMITTED','NO PREFERENCE','UNINSTRUCTED'] or row['party3'] not in ['D','R']:
        primary.loc[idx,'edge1'] = row['pct1']-1.0/2 
        primary.loc[idx,'edge2'] = row['pct2']-1.0/2 
    else:
        primary.loc[idx,'edge1'] = row['pct1']-1.0/3
        primary.loc[idx,'edge2'] = row['pct2']-1.0/3
        primary.loc[idx,'edge3'] = row['pct3']-1.0/3

primary = primary.merge(state_abbr, how = 'left', left_on = 'state', right_on = 'State')
del primary['State']
dem_primary = primary[primary['party1']=='D']
rep_primary = primary[primary['party1']=='R']
dem_primary.head()

,state,county,1st,2nd,3rd,votes1,votes2,votes3,pct1,pct2,pct3,party1,party2,party3,primary_date,primary_type,edge1,edge2,edge3,Abbreviation
0,Alabama,Autauga County,H. CLINTON,B. SANDERS,UNCOMMITTED,2387.0,544.0,42.0,0.800,0.182,0.014,D,D,D,03/01/16,Open primary,0.111667,-0.128333,-0.185333,AL
1,Alabama,Baldwin County,H. CLINTON,B. SANDERS,UNCOMMITTED,5290.0,2694.0,149.0,0.647,0.329,0.018,D,D,D,03/01/16,Open primary,0.135667,-0.140333,-0.163333,AL
2,Alabama,Barbour County,H. CLINTON,B. SANDERS,UNCOMMITTED,2567.0,222.0,33.0,0.906,0.078,0.012,D,D,D,03/01/16,Open primary,0.167667,-0.154333,-0.187333,AL
3,Alabama,Bibb County,H. CLINTON,B. SANDERS,UNCOMMITTED,942.0,246.0,46.0,0.755,0.197,0.037,D,D,D,03/01/16,Open primary,0.160667,-0.078333,-0.221333,AL
4,Alabama,Blount County,H. CLINTON,B. SANDERS,UNCOMMITTED,564.0,395.0,56.0,0.551,0.386,0.055,D,D,D,03/01/16,Open primary,0.153667,-0.089333,-0.211333,AL


In [20]:
primary[primary['Abbreviation']=='KS']

,state,county,1st,2nd,3rd,votes1,votes2,votes3,pct1,pct2,pct3,party1,party2,party3,primary_date,primary_type,edge1,edge2,edge3,Abbreviation
2837,Kansas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03/05/16,Closed caucus,NaN,NaN,NaN,KS
5547,Kansas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03/05/16,Closed caucus,NaN,NaN,-0.274333,KS


In [16]:
county_dems = dem_primary.merge(county_fact, how = 'left', left_on = ['Abbreviation', 'county'], right_on = ['state_abbreviation','area_name'])
county_dems = county_dems[county_dems['state']!='Minnesota']
county_dems = county_dems[~county_dems[['state','county']].duplicated()]
county_dems.to_csv(r'.\stitched_data\county_dems.csv')

In [18]:
tmp = county_dems['state']
tmp2 = tmp[~tmp.duplicated()]
len(tmp2)


48

In [21]:
county_reps = rep_primary.merge(county_fact, how = 'left', left_on = ['Abbreviation', 'county'], right_on = ['state_abbreviation','area_name'])
county_reps = county_reps[county_reps['state']!='Minnesota']
county_reps = county_reps[~county_reps[['state','county']].duplicated()]
county_reps.to_csv(r'.\stitched_data\county_reps.csv')